<a href="https://colab.research.google.com/github/aavisalunkhe/fineTunedModel/blob/main/trainedModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers dataset
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.0/233.0 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 35.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 6.4 MB/s eta 0:00:00
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 2.0.32
    Uninstalling SQLAlchemy-2.0.32:
      Successfully uninstalled SQLAlchemy-2.0.32
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython-sql 0.5.0 requires sqlalchemy>=2.0, but you have sqlalchemy 1.4.53 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.8 MB/s eta 0:00:00

In [ ]:
from transformers import BertTokenizerFast, BertForTokenClassification
from transformers import Trainer, TrainingArguments

model_name= "bert-base-uncased"
#a model somewhere betn distilbert and large bert, offers better coverage(?) than distil and more efficieny and speed than large

tokenizer= BertTokenizerFast.from_pretrained(model_name)
#initializing tokenizer using pre-trained bert model bert-base-uncased, ready to break the input into smaller units/ tokens

model= BertForTokenClassification.from_pretrained(model_name, num_labels=9)
#initialize bert model for token classification
#adjust num_labels based on your specific entities, num_labels=n indicates that the model is being fine-tuned for a classification task with n possible labels (e.g., sensitive vs. non-sensitive)
#ner_tags: a list of classification labels (int). Full tagset with indices:
#{'O': 0, 'B-PER': 1, 'I-PER': 2, 'B-ORG': 3, 'I-ORG': 4, 'B-LOC': 5, 'I-LOC': 6, 'B-MISC': 7, 'I-MISC': 8}

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from datasets import load_dataset
dataset= load_dataset("conll2003")

The repository for conll2003 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/conll2003.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/14041 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3250 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3453 [00:00<?, ? examples/s]

In [ ]:
def tokenize_and_align_labels(examples):
#tokenizes the input text and aligns the labels with the tokenized output. handles conversion of word-level labels to token-level labels
    tokenized_inputs= tokenizer(
        examples["tokens"],
        truncation=True,
        is_split_into_words=True,
        padding="max_length",
        max_length=128,
    )
    #truncation ensures that the text is truncated to fit within the model’s maximum token length and is_split_into_word= true tells the tokenizer that the input is already split into word-level tokens
    labels= []
    for i, label in enumerate(examples[f"ner_tags"]):
    #enumerate is used to iterate over the ner_tags from the examples dictionary, providing both the index and the value of each item in the list
        word_ids= tokenized_inputs.word_ids(batch_index=i)
        #attaches the original word id to the token id
        previous_word_idx= None
        label_ids= []
        for word_idx in word_ids:
            if word_idx is None:
             # if there was no original id (like in case of [pad]) it attaches none to the token id
                label_ids.append(-100)
            elif word_idx!= previous_word_idx:
            #previous_word_idx ensures that each token is only assigned a label if it corresponds to a new word.
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx= word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"]= labels
    return tokenized_inputs

tokenized_datasets = dataset.map(tokenize_and_align_labels, batched=True)
#.map is basically a for loop alternative, it parses through dataset and performs tokenize_and_align_labels taking examples in batches (by batched= True, this ensures that the system is efficient and scalable) rather than one at a time
#applies the function to the dataset, ensuring that each token is correctly aligned with its label.


Map:   0%|          | 0/14041 [00:00<?, ? examples/s]

Map:   0%|          | 0/3250 [00:00<?, ? examples/s]

Map:   0%|          | 0/3453 [00:00<?, ? examples/s]

In [ ]:
training_args = TrainingArguments(
    output_dir="./results", #tells where the model checkpoints and results will be saved
    evaluation_strategy="epoch",#sets how often the model is evaluated during training. here, after each epoch
    learning_rate=2e-5,#common learning rate for bert
    per_device_train_batch_size=8,#no of samples per batch on each device (like GPU)
    num_train_epochs=3,#no of times to iterate over the training data
    weight_decay=0.01,#regularization term to prevent overfitting, set to 0.01.
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.063900,0.049011
2,0.029300,0.053244
3,0.016400,0.052605


TrainOutput(global_step=5268, training_loss=0.051712255950397676, metrics={'train_runtime': 1518.2471, 'train_samples_per_second': 27.744, 'train_steps_per_second': 3.47, 'total_flos': 2751824963545344.0, 'train_loss': 0.051712255950397676, 'epoch': 3.0})

In [ ]:
trainer.evaluate()

Epoch,Training Loss,Validation Loss
0,No log,0.052795


{'eval_loss': 0.05279458314180374}